<a href="https://colab.research.google.com/github/jytan023/data-collection-dump/blob/main/Tweet_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook to save tweets from knowledgeable individuals

In [ ]:
!pip install snscrape
!pip install gspread==5.7.2
!pip3 install --upgrade git+https://github.com/JustAnotherArchivist/snscrape.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 kB 2.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: gspread
    Found existing installation: gspread 3.4.2
    Uninstalling gspread-3.4.2:
      Successfully uninstalled gspread-3.4.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/JustAnotherArchivist/snscrape.git to /tmp/pip-req-build-u3gy0ccc
  Running command git clone --filter=blob:none --quiet https://github.com/JustAnotherArchivist/snscrape.git /tmp/pip-req-build-u3gy0ccc
  Resolved https://github.com/JustAnotherArchivist/snscrape.git to commit b76f485208016f337ce36a3c6c0c6573502693a7
  Installing build dependencies ... done
 

In [ ]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

current_df = pd.DataFrame([], columns=['username','text', 'date','convoid','id','likes','retweets','replies','media','hashtags'])
sheets = gc.open('Twitter Archive').worksheets()
current_df = pd.DataFrame()
for sheet in sheets:
    current_df = pd.concat([current_df, pd.DataFrame(sheet.get_all_records(numericise_ignore=[4,5]))])

In [ ]:
# for sheet in sheets:
#   sheet.format("D", { "numberFormat": { "type": "TEXT" }})
#   sheet.format("E", { "numberFormat": { "type": "TEXT" }})


In [ ]:
current_df.head()

,username,text,date,convoid,id,likes,retweets,replies,media,hashtags,image
0,quant_arb,@nayshins Only slander if it’s not true,2023-06-04 15:20:18+00:00,1665373628530147328,1665377946763141121,0,0,0,None,None,
1,quant_arb,Day in the life of a quant\n\n1. Stare at inco...,2023-06-04 15:03:08+00:00,1665373628530147328,1665373628530147328,77,1,6,None,None,
2,quant_arb,https://t.co/WHMxn913ON,2023-06-04 14:22:36+00:00,1665363427961847808,1665363427961847808,21,0,6,[Photo(previewUrl='https://pbs.twimg.com/media...,None,
3,quant_arb,@contang0 Looks like cow shit after feeding sa...,2023-06-04 13:30:07+00:00,1665349709370630144,1665350217779093504,0,0,2,None,None,
4,quant_arb,Follow for more health tips,2023-06-04 13:28:21+00:00,1665349709370630144,1665349772041920514,14,0,2,None,None,


In [ ]:
since_date = max(current_df['date'])[:10]
# since_date = '2023-03-25'
# since_date = '2000-01-01'
# Define the Twitter handle of the user to scrape
twitter_handles1 = [ 'quant_arb', 'pyquantnews','VolatilitySwan','Vertox_DF', 'chameleon_jeff', 'therobotjames','QuantumHoneybee','hasantoxr']
twitter_handles2 =  ['abacusai','TheRundownAI','AlexIsBuilding','svpino','NT_Quant','Uncentive1','JKevin2011','nik_algo','Gingfacekillah','quant_porn']
twitter_handles3 =['abetrade','chanep','ltrd_','choffstein','WifeyAlpha','nadogoyard','IAmPascio','FabiusMercurius','heyBarsee','Analytic_Trader']

# Define the search query to scrape tweets from the user's profile
# query = f'from:{twitter_handle}'
query1 = ' OR '.join(f'from:{handle}' for handle in twitter_handles1) + f' since:{since_date}'
query2 = ' OR '.join(f'from:{handle}' for handle in twitter_handles2) + f' since:{since_date}'
query3 = ' OR '.join(f'from:{handle}' for handle in twitter_handles3) + f' since:{since_date}'

In [ ]:
# Scrape the tweets using snscrape and store the results in a list
def get_tweets(query):
  tweets = []
  for tweet in sntwitter.TwitterSearchScraper(query).get_items():
      author_username = tweet.user.username
      # author_followers = tweet.user.followersCount
      # author_following = tweet.user.friendsCount
      # author_verified = tweet.user.verified
      tweet_text = tweet.rawContent
      tweet_date = tweet.date
      tweet_id = tweet.id
      tweet_likes = tweet.likeCount
      tweet_retweets = tweet.retweetCount
      tweet_replies = tweet.replyCount
      tweet_media = tweet.media
      tweet_hashtags = tweet.hashtags
      tweet_convoid = tweet.conversationId
      tweets.append([author_username, tweet_text, tweet_date, tweet_convoid, tweet_id, tweet_likes, tweet_retweets, tweet_replies, tweet_media, tweet_hashtags])
      # Convert the list of tweets to a Pandas DataFrame

  df = pd.DataFrame(tweets, columns=['username','text', 'date','convoid','id','likes','retweets','replies','media','hashtags'])
  print(query)
  return(df)

df = pd.concat([get_tweets(query1),get_tweets(query2),get_tweets(query3)])

# Print the DataFrame
print(df.head())

from:quant_arb OR from:pyquantnews OR from:VolatilitySwan OR from:Vertox_DF OR from:chameleon_jeff OR from:therobotjames OR from:QuantumHoneybee OR from:hasantoxr since:2023-06-04


from:abacusai OR from:TheRundownAI OR from:AlexIsBuilding OR from:svpino OR from:NT_Quant OR from:Uncentive1 OR from:JKevin2011 OR from:nik_algo OR from:Gingfacekillah OR from:quant_porn since:2023-06-04


from:abetrade OR from:chanep OR from:ltrd_ OR from:choffstein OR from:WifeyAlpha OR from:nadogoyard OR from:IAmPascio OR from:FabiusMercurius OR from:heyBarsee OR from:Analytic_Trader since:2023-06-04
        username                                               text  \
0      hasantoxr  That's a wrap! Hope this was helpful!  \n\nIf ...   
1      hasantoxr  Special Bonus:\n\nJoin my free weekly newslett...   
2      quant_arb  @zackvoell “Fuck your puts, fuck your calls, G...   
3      quant_arb                CZ is now a registered SEC offender   
4  therobotjames  @Bonecondor reading all your tweets in a new j...   

                       date              convoid                   id  likes  \
0 2023-06-06 13:40:05+00:00  1666055832100806662  1666077500802490374      3   
1 2023-06-06 13:38:09+00:00  1666055832100806662  1666077016989536259      4   
2 2023-06-06 13:28:11+00:00  1666073812842995714  1666074506245357568      4   
3 2023-06-06 12:59:43+00:00  1666067345406676995  16

In [ ]:
twitter_handles = twitter_handles1 + twitter_handles2 + twitter_handles3
print(twitter_handles)

['quant_arb', 'pyquantnews', 'VolatilitySwan', 'Vertox_DF', 'chameleon_jeff', 'therobotjames', 'QuantumHoneybee', 'hasantoxr', 'abacusai', 'TheRundownAI', 'AlexIsBuilding', 'svpino', 'NT_Quant', 'Uncentive1', 'JKevin2011', 'nik_algo', 'Gingfacekillah', 'quant_porn', 'abetrade', 'chanep', 'ltrd_', 'choffstein', 'WifeyAlpha', 'nadogoyard', 'IAmPascio', 'FabiusMercurius', 'heyBarsee', 'Analytic_Trader']


In [ ]:
df = pd.concat([df,current_df])
df = df.astype(str)
df = df.reset_index().drop('index',axis = 1)
df = df.drop_duplicates(subset=['username','text','date','convoid','id','media'], keep='first').reset_index().drop('index',axis = 1)

df

,username,text,date,convoid,id,likes,retweets,replies,media,hashtags,image
0,hasantoxr,That's a wrap! Hope this was helpful! \n\nIf ...,2023-06-06 13:40:05+00:00,1666055832100806662,1666077500802490374,3,0,0,None,None,nan
1,hasantoxr,Special Bonus:\n\nJoin my free weekly newslett...,2023-06-06 13:38:09+00:00,1666055832100806662,1666077016989536259,4,1,1,[Photo(previewUrl='https://pbs.twimg.com/media...,None,nan
2,quant_arb,"@zackvoell “Fuck your puts, fuck your calls, G...",2023-06-06 13:28:11+00:00,1666073812842995714,1666074506245357568,4,0,0,None,None,nan
3,quant_arb,CZ is now a registered SEC offender,2023-06-06 12:59:43+00:00,1666067345406676995,1666067345406676995,25,0,2,None,None,nan
4,therobotjames,@Bonecondor reading all your tweets in a new j...,2023-06-06 12:48:52+00:00,1666062380701298688,1666064611597230081,8,0,1,None,None,nan
...,...,...,...,...,...,...,...,...,...,...,...
288257,Analytic_Trader,https://t.co/6GmlpVDwUO,2016-09-14 17:02:40+00:00,776103888021811200,776103888021811200,0,0,0,[Photo(previewUrl='https://pbs.twimg.com/media...,None,
288258,Analytic_Trader,https://t.co/2PJm9XqN3x,2016-09-14 06:12:14+00:00,775940199230472192,775940199230472192,0,0,0,[Photo(previewUrl='https://pbs.twimg.com/media...,None,
288259,Analytic_Trader,https://t.co/4PPkEhkmiC,2016-09-12 15:52:02+00:00,775361334489837568,775361334489837568,0,0,0,None,None,
288260,Analytic_Trader,https://t.co/8KkqyrD8aa,2016-09-08 22:07:15+00:00,774006210131558403,774006210131558403,0,0,0,None,None,


In [ ]:
def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

# def path_to_image_html(path):
#     return '<img src="'+ path + '" width="60" >'

def path_to_image_html(path):
    return '=image('+'"'+ path +'"'+ ',4,200,200)'

df['image']=df['media'].apply(lambda x: path_to_image_html(find_between(x,"fullUrl="+"'","'"+", altText=")))

In [ ]:
twitter_handles = twitter_handles1 + twitter_handles2 + twitter_handles3
# Open our new sheet and add some data.
for user in twitter_handles:
  try: 
    worksheet = gc.open('Twitter Archive').add_worksheet(title=user, rows=100, cols=20)
    print(user, ' worksheet created')
  except:
    worksheet = gc.open('Twitter Archive').worksheet(user)
    # gc.open('Twitter Archive').del_worksheet(worksheet)
    # worksheet = gc.open('Twitter Archive').add_worksheet(title=user, rows=100, cols=20)
    print(user, ' worksheet existed')
  worksheet = gc.open('Twitter Archive').worksheet(user)
  all_processed_data_columns = df.columns.to_list()
  all_processed_data_list = df[df['username']==user].to_numpy().tolist()
  all_processed_data_all = [all_processed_data_columns] + all_processed_data_list
  # worksheet.append_rows(all_processed_data_all)
  worksheet.update(None,all_processed_data_all,value_input_option='USER_ENTERED')


quant_arb  worksheet existed
pyquantnews  worksheet existed
VolatilitySwan  worksheet existed
Vertox_DF  worksheet existed
chameleon_jeff  worksheet existed
therobotjames  worksheet existed
QuantumHoneybee  worksheet existed
hasantoxr  worksheet existed
abacusai  worksheet existed
TheRundownAI  worksheet existed
AlexIsBuilding  worksheet existed
svpino  worksheet existed
NT_Quant  worksheet existed
Uncentive1  worksheet existed
JKevin2011  worksheet existed
nik_algo  worksheet existed
Gingfacekillah  worksheet existed
quant_porn  worksheet existed
abetrade  worksheet existed
chanep  worksheet existed
ltrd_  worksheet existed
choffstein  worksheet existed
WifeyAlpha  worksheet existed
nadogoyard  worksheet existed
IAmPascio  worksheet existed
FabiusMercurius  worksheet existed
heyBarsee  worksheet existed
Analytic_Trader  worksheet existed


## Convert link to image

In [ ]:
  for user in twitter_handles:
    sheet_id = gc.open('Twitter Archive').worksheet(user).id
    reqs = [
        {
            "copyPaste": {
                "source": {
                    "sheetId": sheet_id,
                    "startRowIndex": 1,
                    # "endRowIndex": 2,
                    "startColumnIndex": 11,
                    "endColumnIndex": 11,
                },
                "destination": {
                    "sheetId": sheet_id,
                    "startRowIndex": 1,
                    # "endRowIndex": 2,
                    "startColumnIndex": 12,
                    "endColumnIndex": 12,
                },
                "pasteType": "PASTE_VALUES",
                "pasteOrientation": "NORMAL",
            }
        }
    ]
    res = gc.open('Twitter Archive').batch_update({"requests": reqs})
    print(user)

quant_arb
pyquantnews
VolatilitySwan
Vertox_DF
chameleon_jeff
therobotjames
QuantumHoneybee
hasantoxr
abacusai
TheRundownAI
AlexIsBuilding
svpino
NT_Quant
Uncentive1
JKevin2011
nik_algo
Gingfacekillah
quant_porn
abetrade
chanep
ltrd_
choffstein
WifeyAlpha
nadogoyard
IAmPascio
FabiusMercurius
heyBarsee
Analytic_Trader


In [ ]:
def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

# def path_to_image_html(path):
#     return '<img src="'+ path + '" width="60" >'

def path_to_image_html(path):
    return '=image('+'"'+ path +'"'+ ')'

current_df['image']=current_df['media'].apply(lambda x: path_to_image_html(find_between(x,"fullUrl="+"'","'"+", altText=")))

In [ ]:
current_df['image_link']=current_df['media'].apply(lambda x: "=image("+'"'+find_between(x,"fullUrl="+"'","'"+", altText=")+'"'+")")

In [ ]:
twitter_handles = twitter_handles1 + twitter_handles2 + twitter_handles3
# Open our new sheet and add some data.
for user in twitter_handles:
  try: 
    worksheet = gc.open('Twitter Archive').add_worksheet(title=user, rows=100, cols=20)
    print(user, ' worksheet created')
  except:
    worksheet = gc.open('Twitter Archive').worksheet(user)
    # gc.open('Twitter Archive').del_worksheet(worksheet)
    # worksheet = gc.open('Twitter Archive').add_worksheet(title=user, rows=100, cols=20)
    print(user, ' worksheet existed')
  worksheet = gc.open('Twitter Archive').worksheet(user)
  all_processed_data_columns = df.columns.to_list()
  all_processed_data_list = df[df['username']==user].to_numpy().tolist()
  all_processed_data_all = [all_processed_data_columns] + all_processed_data_list
  # worksheet.append_rows(all_processed_data_all)
  worksheet.update(None,all_processed_data_all,value_input_option='USER_ENTERED')

quant_arb  worksheet existed
pyquantnews  worksheet existed
VolatilitySwan  worksheet existed
Vertox_DF  worksheet existed
chameleon_jeff  worksheet existed
therobotjames  worksheet existed
QuantumHoneybee  worksheet existed
hasantoxr  worksheet existed
abacusai  worksheet existed
TheRundownAI  worksheet existed
AlexIsBuilding  worksheet existed
svpino  worksheet existed
NT_Quant  worksheet existed
Uncentive1  worksheet existed
JKevin2011  worksheet existed
nik_algo  worksheet existed
Gingfacekillah  worksheet existed
quant_porn  worksheet existed
abetrade  worksheet existed
chanep  worksheet existed
ltrd_  worksheet existed
choffstein  worksheet existed
WifeyAlpha  worksheet existed
nadogoyard  worksheet existed
IAmPascio  worksheet existed
FabiusMercurius  worksheet existed
heyBarsee  worksheet existed
Analytic_Trader  worksheet existed


In [ ]:
current_df

,username,text,date,convoid,id,likes,retweets,replies,media,hashtags,image,image_link
0,quant_arb,@nayshins Only slander if it’s not true,2023-06-04 15:20:18+00:00,1665373628530147328,1665377946763141121,0,0,0,None,None,"=image("""")","=image("""")"
1,quant_arb,Day in the life of a quant\n\n1. Stare at inco...,2023-06-04 15:03:08+00:00,1665373628530147328,1665373628530147328,77,1,6,None,None,"=image("""")","=image("""")"
2,quant_arb,https://t.co/WHMxn913ON,2023-06-04 14:22:36+00:00,1665363427961847808,1665363427961847808,21,0,6,[Photo(previewUrl='https://pbs.twimg.com/media...,None,"=image(""https://pbs.twimg.com/media/FxyPMzsWcA...","=image(""https://pbs.twimg.com/media/FxyPMzsWcA..."
3,quant_arb,@contang0 Looks like cow shit after feeding sa...,2023-06-04 13:30:07+00:00,1665349709370630144,1665350217779093504,0,0,2,None,None,"=image("""")","=image("""")"
4,quant_arb,Follow for more health tips,2023-06-04 13:28:21+00:00,1665349709370630144,1665349772041920514,14,0,2,None,None,"=image("""")","=image("""")"
...,...,...,...,...,...,...,...,...,...,...,...,...
2354,Analytic_Trader,https://t.co/6GmlpVDwUO,2016-09-14 17:02:40+00:00,776103888021811200,776103888021811200,0,0,0,[Photo(previewUrl='https://pbs.twimg.com/media...,None,"=image(""https://pbs.twimg.com/media/CsVGXW-WEA...","=image(""https://pbs.twimg.com/media/CsVGXW-WEA..."
2355,Analytic_Trader,https://t.co/2PJm9XqN3x,2016-09-14 06:12:14+00:00,775940199230472192,775940199230472192,0,0,0,[Photo(previewUrl='https://pbs.twimg.com/media...,None,"=image(""https://pbs.twimg.com/media/CsSxfSqUEA...","=image(""https://pbs.twimg.com/media/CsSxfSqUEA..."
2356,Analytic_Trader,https://t.co/4PPkEhkmiC,2016-09-12 15:52:02+00:00,775361334489837568,775361334489837568,0,0,0,None,None,"=image("""")","=image("""")"
2357,Analytic_Trader,https://t.co/8KkqyrD8aa,2016-09-08 22:07:15+00:00,774006210131558403,774006210131558403,0,0,0,None,None,"=image("""")","=image("""")"
